In [1]:
import requests
import collections
import re
from datetime import date
import datetime,time
from dateutil.rrule import rrule, DAILY, HOURLY
import pandas as pd
from utils import *
import sys

#sys.path.append('/usr/local/lib/python2.7/site-packages/')

In [17]:
import logging
logger=logging.getLogger()
logging.basicConfig(filename='./log.log',level=logging.WARNING)
#logger.setLevel(logging.WARNING)

Check quota [here](https://console.developers.google.com/project/childmarriage-1019/apiui/apiview/youtube/quotas)

In [18]:
taxonomyWords=['child marriage','child bride','underage bride','teen bride','forced marriage','early marriage']
taxonomyWords.extend([u'बाल विवाह',u'बालिका वधु',u'नाबालिक विवाह',u'नाबालिक वधु',u'किशोर वधु',u'किशोर बीवी',u'जबरन शादी',u'जबरन विवाह'])
taxonomyWords=map(lambda x:'"'+x+'"',taxonomyWords)
FULLQUERY='|'.join(taxonomyWords)

In [19]:
FULLQUERY

u'"child marriage"|"child bride"|"underage bride"|"teen bride"|"forced marriage"|"early marriage"|"\u092c\u093e\u0932 \u0935\u093f\u0935\u093e\u0939"|"\u092c\u093e\u0932\u093f\u0915\u093e \u0935\u0927\u0941"|"\u0928\u093e\u092c\u093e\u0932\u093f\u0915 \u0935\u093f\u0935\u093e\u0939"|"\u0928\u093e\u092c\u093e\u0932\u093f\u0915 \u0935\u0927\u0941"|"\u0915\u093f\u0936\u094b\u0930 \u0935\u0927\u0941"|"\u0915\u093f\u0936\u094b\u0930 \u092c\u0940\u0935\u0940"|"\u091c\u092c\u0930\u0928 \u0936\u093e\u0926\u0940"|"\u091c\u092c\u0930\u0928 \u0935\u093f\u0935\u093e\u0939"'

In [20]:
QUERY=re.escape('"child marriage"|"child bride"')
from secrets import KEY

diff=1

In [21]:
def getVideoParts(vData):
    title=res['items'][0]['snippet'].get('title')
    time=res['items'][0]['snippet'].get('publishedAt')
    tags=res['items'][0]['snippet'].get('tags')
    lang=res['items'][0]['snippet'].get('defaultAudioLanguage')
    description=res['items'][0]['snippet'].get('description')
    commentCount=viewCount=favouriteCount=dislikeCount=likeCount=None
    
    stats=res['items'][0].get('statistics')

    commentCount=stats.get('commentCount')
    viewCount=stats.get('viewCount')
    favouriteCount=stats.get('favoriteCount')
    dislikeCount=stats.get('dislikeCount')
    likeCount=stats.get('likeCount')
    
    return time,title,tags,lang,description,commentCount,viewCount,favouriteCount,dislikeCount,likeCount

In [22]:
def getCommentsFromVideo(vid):
    requestString='https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&videoId='+vid+'&key='+KEY

    res=requests.get(requestString)
    commentData=res.json()

    if res.status_code==403:
        # Comments disabled
        logging.warning('Comments error %d,%s' % (res.status_code,vid))
        logging.warning(commentData)
        return [],None
    
    isMore=False
    
    nextToken=commentData.get('nextPageToken')
    
    if nextToken:
        if len(nextToken)>0:
            logging.warning('!!!!More than 100 comments....')
            isMore=True
    
    try:
        logging.info('Got %d comments' % (len(commentData['items'])))
    except:
        logging.warning('!!!!ERROR',commentData.keys())
        logging.warning('%d' % res.status_code)
        logging.warning(commentData)
    
    return commentData['items'],isMore

In [23]:
def getCountryFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('country')
    else:
        return None

In [24]:
def getDescriptionFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('description')
    else:
        return None

In [25]:
def getTitleFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('title')
    else:
        return None

In [26]:
def getVideoInfoFromVideo(vid):

        requestString='https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id='+vid+'&key='+KEY

        logging.info('QUERYING VIDEO %s' % vid)

        d=requests.get(requestString)

        return d.json()

In [27]:
def getChannelInfoFromChannel(cid):
    requestString='https://www.googleapis.com/youtube/v3/channels?part=snippet&id='+cid+'&key='+KEY

    logging.info('QUERYING CHANNEL'+cid)

    d=requests.get(requestString)

    return d.json()

In [28]:
def getRepliesFromComment(commentId):
    requestString='https://www.googleapis.com/youtube/v3/comments?part=snippet&maxResults=100&parentId='+commentId+\
    '&key='+KEY
    
    isMore=False
    
    d=requests.get(requestString)
    
    returnData=d.json()
    
    nextToken=returnData.get('nextPageToken')
    
    if nextToken:
        if len(nextToken)>0:
            logging.warning('!!!!More than 100 comments....')
            isMore=True
    
    return returnData,isMore

In [ ]:
%%time
nDayDiff=7*4
# Look back n days from start date

diff=1
# Start date is yesterday

now=datetime.datetime.now()

endDate=date(now.year,now.month,now.day)
# Throw away hour, minute, second

endDate=endDate-datetime.timedelta(days=diff)
endDateTemp=endDate
# Look back diff days

startDate=endDate-datetime.timedelta(days=nDayDiff)
startDateTemp=startDate

endDate=endDate.strftime("%Y-%m-%dT%H:%M:%SZ")
startDate=startDate.strftime("%Y-%m-%dT%H:%M:%SZ")

startDates=[startDate]
endDates=[endDate]

logging.info('%s - %s' %  (' '.join(startDates),' '.join(endDates)))

startDates=[d for d in rrule(DAILY,dtstart=startDateTemp,until=endDateTemp-datetime.timedelta(days=1))]
startDates=[d.strftime("%Y-%m-%dT%H:%M:%SZ") for d in startDates]

logging.info('Start dates %s' % (' '.join(startDates)))

vIdList=[]
vTimeList=[]
vTitleList=[]
vLangList=[]
vTagList=[]
vDescriptionList=[]
vCommentCountList=[]
vViewCountList=[]
vFavouriteCountList=[]
vDislikeCountList=[]
vLikeCountList=[]

cIdList=[]
cCountryList=[]
cDescriptionList=[]
cTitleList=[]

commentIdList=[]
commentVideoIdList=[]
commentTextList=[]
commentUserIdList=[]
commentLikesList=[]
commentTimeList=[]

# commentCount:0 viewCount:4 favoriteCount:0 dislikeCount:0 likeCount:0

replyTextList=[]
replyCommentIdList=[]
replyTimeList=[]
replyAuthorIdList=[]
replyLikeCountList=[]

##############
for start,end in zip(startDates[0:-1],startDates[1:]):

    requestString='https://www.googleapis.com/youtube/v3/search?part=snippet&q='+FULLQUERY+'&key='+KEY+'&maxResults=50&type=video&publishedBefore='+end+'&publishedAfter='+start

    print 'QUERYING API IN RANGE',start,end
    logging.info('QUERYING API IN RANGE %s - %s' % (start,end))
    
    d=requests.get(requestString)
    logging.info('Search return code %d' % d.status_code)

    data=d.json()
    
    logging.info('Got %d videos' % (len(data['items'])))

    videoIds=[]

    for v,video in enumerate(data['items']):
        
        ############################################
        # Video stuff

        videoId=video[u'id']['videoId']
        vIdList.append(videoId)
        
        videoIds.append(videoId)
        videoData=getVideoData(videoId)

        logging.info('Video ID %s' % videoId)
        logging.info('Video keys %s' % (videoData['items'][0].keys()))

        res=getVideoInfoFromVideo(videoData['items'][0]['id'])
        channelId=res['items'][0]['snippet']['channelId']

        vTime,vTitle,vTags,vLang,vDescription,vCommentCount,vViewCount,vFavouriteCount,vDislikeCount,vLikeCount=\
        getVideoParts(res['items'][0])
        
        logging.info('Video Title: %s' % vTitle)
        vTitleList.append(vTitle)
        
        logging.info('Video time: %s' % vTime)
        vTimeList.append(vTime)
        
        logging.info('Video tags: %s' % vTags)
        vTagList.append(vTags)
        
        if vLang:logging.info('Video lang: %s' % vLang)
        vLangList.append(vLang)
        
        if False:print 'Video Description:',vDescription
        vDescriptionList.append(vDescription)
        
        logging.info('Video Stats %s' % ' '.join([k+':'+v for k,v in zip(['comments','views','favs','dislike','like'],[vCommentCount,vViewCount,vFavouriteCount,vDislikeCount,vLikeCount]) if v]))
        vCommentCountList.append(vCommentCount)
        vViewCountList.append(vViewCount)
        vFavouriteCountList.append(vFavouriteCount)
        vDislikeCountList.append(vDislikeCount)
        vLikeCountList.append(vLikeCount)
        
        commentThreads,isMoreComments=getCommentsFromVideo(videoId)
        
        for thread in commentThreads:
            
            commentId=thread['snippet']['topLevelComment']['id']
            commentIdList.append(commentId)
            
            commentVideoId=thread['snippet']['topLevelComment']['snippet']['videoId']
            commentVideoIdList.append(commentVideoId)
            
            commentText=thread['snippet']['topLevelComment']['snippet']['textDisplay']
            commentTextList.append(commentText)
            
            commentTime=thread['snippet']['topLevelComment']['snippet']['publishedAt']
            commentTimeList.append(commentTime)
            
            try:commentUser=thread['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
            except:commentuser=None
            
            commentUserIdList.append(commentUser)
            
            commentLikes=thread['snippet']['topLevelComment']['snippet']['likeCount']
            commentLikesList.append(commentLikes)
                    
            if thread['snippet']['totalReplyCount']>0:
                replies,isMoreReplies=getRepliesFromComment(commentId)
                logging.info('%d replies' % (len(replies)))
                
                for reply in replies['items']:
                    replyText=reply['snippet']['textDisplay']
                    replyTextList.append(replyText)
                    
                    replyCommentId=reply['snippet']['parentId']
                    replyCommentIdList.append(replyCommentId)
                    
                    replyTime=reply['snippet']['publishedAt']
                    replyTimeList.append(replyTime)
                    
                    replyAuthorId=reply['snippet']['authorChannelId']['value']
                    replyAuthorIdList.append(replyAuthorId)
                    
                    replyLikeCount=reply['snippet']['likeCount']
                    replyLikeCountList.append(replyLikeCount)
                
        ############################################
        # Channel stuff
        
        info=getChannelInfoFromChannel(channelId)
        cIdList.append(channelId)

        cCountry=getCountryFromInfo(info)
        logging.info('Channel Country: %s' % cCountry)
        cCountryList.append(cCountry)
        
        cDescription=getDescriptionFromInfo(info)
        if False:print 'Channel Description:',cDescription
        cDescriptionList.append(cDescription)
        
        cTitle=getTitleFromInfo(info)
        logging.info('Channel Title %s' % cTitle)
        cTitleList.append(cTitle)
        
        logging.info('\n')
        
        assert len(vIdList)==len(vTitleList)==len(vTagList)==len(vLangList)==len(vDescriptionList)==len(vViewCountList)\
        ==len(vFavouriteCountList)==len(vDislikeCountList)==len(vLikeCountList)==len(vTimeList)\
        ,'Video data mismatched'
        
        assert len(commentIdList)==len(commentLikesList)==len(commentTextList)==len(commentTimeList)\
        ==len(commentUserIdList)==len(commentVideoIdList)\
        ,'Comment data mismatched'
        
        assert len(replyAuthorIdList)==len(replyCommentIdList)==len(replyLikeCountList)==len(replyTextList)\
        ==len(replyTimeList)\
        ,'Reply data mismatched'
        
    logging.warning('Stats: %d videos %d comments %d replies' %(len(vIdList),len(commentIdList),len(replyCommentId)))

videoDf=pd.DataFrame(data={'id':vIdList,'title':vTitleList,'tags':vTagList,'lang':vLangList,'description':vDescriptionList,\
                'views':vViewCountList,'favourites':vFavouriteCountList,'dislikes':vDislikeCountList,\
                'likes':vLikeCountList,'channelId':cIdList,'channelCountry':cCountryList,\
                'channelDescription':cDescriptionList,'channelTitle':cTitleList},index=vTimeList)       
      
commentDf=pd.DataFrame(data={'text':commentTextList,'videoId':commentVideoIdList,'id':commentIdList,\
                             'likes':commentLikesList,'user':commentUserIdList},index=commentTimeList)

replyDf=pd.DataFrame(data={'text':replyTextList,'likes':replyLikeCountList,'author':replyAuthorIdList,\
                           'parent':replyCommentIdList},index=replyTimeList)

QUERYING API IN RANGE 2015-07-07T00:00:00Z 2015-07-08T00:00:00Z
QUERYING API IN RANGE

In [30]:
print replyDf.shape,videoDf.shape,commentDf.shape

(15, 4) (300, 13) (31, 5)


In [31]:
commentDf

,id,likes,text,user,videoId
2015-07-28T18:15:30.043Z,z12fzjhplzjmdpyae04cetex5v30xxgjb30,0,,UCzOI2rEkVs-VQ59Ychhj6Zg,pzo4Sb67-nI
2015-08-03T20:44:50.577Z,z12pzzn5mmrkgpkuw23luj5xcqudvhrtz,0,We quote below from two such reports in Bukhar...,UCzccEsLkSsteKzBeBIZHTmQ,n0o63BUJgZQ
2015-08-03T20:44:03.284Z,z13ijvh4epzsd1jky23luj5xcqudvhrtz,0,And being married to Aisha does not mean that ...,UCzccEsLkSsteKzBeBIZHTmQ,n0o63BUJgZQ
2015-08-01T01:37:06.120Z,z12wgtzretimyt3pk04ch1lijweaxlyosks0k,0,Bravo :))﻿,UCNUwZHhacqS9gNCFF_ZVVdA,n0o63BUJgZQ
2015-07-28T22:06:29.723Z,z13gzn3imuicwjmjz04cfnfb1sycx5rpunc0k,0,,UCJRjdWHT92qbP7JXElBrYyw,Gv2PshcCcoM
2015-06-29T15:43:03.267Z,z133ezirksvyttf3023rybqb0vrwh1oxr04,0,<b>دوره سلامت روح و روان</b><br /><b>جلسه پنجم...,UCJRjdWHT92qbP7JXElBrYyw,4nM7Mb8hffE
2015-06-29T15:18:10.122Z,z13ctn3z4uf2croj404cdhcxdqavx5mhnes,17,<b>دوره سلامت روح و روان</b><br /><b>جلسه پنجم...,UCaIAr5D3Ceiz0kbCrsCvhVw,4nM7Mb8hffE
2015-07-28T16:06:40.305Z,z12kibqr1unazncxa04cdx1wkkuuubywfwg0k,0,,UCugcfsSMqwwtUn64uF4-Oow,sigdlEpXkk8
2015-07-31T08:44:31.619Z,z12sy3nw5yqzejnvd23se1gyqpm1efwwy04,0,Check out this video on YouTube:﻿,UCRx3IIAjw_zK0gq7qWe36bg,eukIEqQ8Hes
2015-07-31T08:40:54.198Z,z12fgtrz0szkylja204cj1c5nuuqc5djnns0k,0,Check out this video on YouTube:﻿,UCRx3IIAjw_zK0gq7qWe36bg,eukIEqQ8Hes
